# Beaver Tutorial 1: Hello World

Welcome to Beaver! This tutorial will walk you through the basics of collaborative data science with **privacy-preserving Twins**.

## What You'll Learn

1. Setting up Beaver and connecting to a session
2. Creating and sharing data with **Twins**
3. Running computations on remote data
4. Requesting and approving private data access

This tutorial will run on a single notebook and take turns being both Data Owner and Data Scientist, however these same commands would normally be run between two different people on either end of a session.

## Step 1: Import Beaver and Create a Context

First, we import `beaver` and create a **context** (`bv`). The context is your gateway to all Beaver functionality - it lets you:
- Decorate functions with `@bv` to make them sendable and runnable on remote data.
- Access datasets and remote variables
- Manage sessions with peers

In [1]:
# import beaver
# beaver.dev()

In [2]:
import beaver

# Create the Beaver context - this auto-detects your identity from the session
bv = beaver.ctx()

🔄 Auto-load replies enabled for kj@kj.dev (polling every 0.5s)


In [3]:
# beaver.show_debug()

## Step 2: Connect to Your Session

When you launch a notebook from a BioVault session, it's automatically given several environment variables which tell beaver what **session** you are in and where to write and read data.

A session is a secure communication channel between you and one or more peers (another researcher).

**Tip:** You can start a session with yourself for testing! This is useful for learning how Beaver works before collaborating with others.

To see these environment variables run:
```python
beaver.show_debug()
```

In [4]:
# Get the active session - this notebook is tied to a specific session
session = bv.active_session()

# View session info
session

🟢 Active session loaded: 49bd18a519de
   Peer: kj@kj.dev


🟢 Session with kj@kj.dev
   ID: 49bd18a519de...
   Role: peer
   Status: active

📥 session.inbox():
   InboxView(/Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/sandbox/kj@kj.dev/datasites/kj@kj.dev/shared/biovault/sessions/49bd18a519de): empty

You can see where the files are on your computer by running:
```python
session.open()
```

In [ ]:
if session.peer != session.owner:
    raise AssertionError(
        "Hello Beaver tutorial won't work when in a session with someone else"
    )

In [ ]:
# session.open()

## Step 3: Browse Available Datasets

In Beaver, data is organized into **datasets** published by different users. You can browse what's available:

In [6]:
# See all available datasets
bv.datasets

Owner,Datasets
kj@kj.dev,genotype
madhava@openmined.org,single_cell
me@madhavajay.com,test


**Tip:** In the BioVault app, go to **Network → Datasets** tab to browse datasets visually. You can copy/paste the import code directly from there!

### Creating a Dataset in BioVault App

You can also create datasets using the BioVault app UI:

1. Go to **Data → New Dataset**
2. Give your dataset a name
3. Add files as **Private** (real data) and **Mock** (synthetic data) Twins
4. Click **Save Dataset**
5. Hit **Publish** to make it available to peers

Once published, your dataset will appear here in the Python API!

To access a specific dataset:

In [7]:
# Access a dataset by owner and name
# Example: dataset = bv.datasets["owner@email.com"]["dataset_name"]

# View what's in the session peer's datasets (if they have any)
print(f"Looking for datasets from: {session.peer}")
try:
    peer_datasets = bv.datasets[session.peer]
    print(peer_datasets)
except KeyError:
    print("No datasets published yet. Let's create one!")

Looking for datasets from: kj@kj.dev
Datasets(owner=kj@kj.dev, datasets=[genotype])


## Step 4: Create Your Own Data (The Twin Pattern)

A **Twin** is the core concept in Beaver. It has two sides:
- **Public side** (mock/synthetic data): Safe to share, used for testing
- **Private side** (real data): Sensitive, requires explicit approval to access

Let's create some example data files first:

In [8]:
import os

# Create a local data folder
data_dir = "./my_data"
os.makedirs(data_dir, exist_ok=True)

# Create MOCK data (synthetic/public - safe to share)
mock_csv = f"{data_dir}/mock_prices.csv"
with open(mock_csv, 'w') as f:
    f.write("item,units,price\n")
    f.write("apple,2,1.40\n")
    f.write("banana,3,0.75\n")
    f.write("orange,1,1.10\n")

# Create REAL data (private - sensitive)
real_csv = f"{data_dir}/real_prices.csv"
with open(real_csv, 'w') as f:
    f.write("item,units,price\n")
    f.write("apple,10,1.50\n")
    f.write("banana,8,0.80\n")
    f.write("orange,12,1.20\n")

print(f"Created mock data: {mock_csv}")
print(f"Created real data: {real_csv}")

Created mock data: ./my_data/mock_prices.csv
Created real data: ./my_data/real_prices.csv


## Step 5: Create a Twin

Now let's wrap this data in a Twin. The Twin keeps track of both versions and ensures the private data is only accessible with explicit approval.

In [9]:
# install pandas
!uv pip install pandas

Audited 1 package in 8ms


In [10]:
import pandas as pd
from beaver import Twin

# Load both datasets
mock_df = pd.read_csv(mock_csv)
real_df = pd.read_csv(real_csv)

# Create a Twin with both sides
prices = Twin(
    public=mock_df,    # Safe to share - used for testing
    private=real_df,   # Sensitive - requires approval
    name="prices",
    # owner=bv.user      # Your identity
)

# View the Twin
prices

⚠️  Twin: prices (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private         item  units  price
0   apple     10    1.5
1  banana...    ← .value uses this
  🌍 Public         item  units  price
0   apple      2   1.40
1  banana...    ✓
  Live: ⚫ Disabled
  IDs: twin=2bfc7aef... private=d0ef9eb3... public=e907e55c...

In [11]:
prices.private

,item,units,price
0,apple,10,1.5
1,banana,8,0.8
2,orange,12,1.2


In [12]:
prices.public

,item,units,price
0,apple,2,1.40
1,banana,3,0.75
2,orange,1,1.10


In [13]:
prices.value

🔒 Using PRIVATE data from Twin 'prices...'


,item,units,price
0,apple,10,1.5
1,banana,8,0.8
2,orange,12,1.2


## Step 6: Publish the Twin as a Remote Variable

To share data with your session peer, publish it as a **remote variable**. Only the **public side** is shared - the private data stays with you!

This nearly identical to the pre-existing datasets however it will only be available inside the session and `remote_vars` can also track live updates.

In [14]:
prices

⚠️  Twin: prices (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private         item  units  price
0   apple     10    1.5
1  banana...    ← .value uses this
  🌍 Public         item  units  price
0   apple      2   1.40
1  banana...    ✓
  Live: ⚫ Disabled
  IDs: twin=2bfc7aef... private=d0ef9eb3... public=e907e55c...

In [15]:
# Publish to session - peer can now see this
session.remote_vars["prices"] = prices


⚠️  Twin 'prices' .public has unloaded data.
   Call .load() to load it, or approve now:
🌍 Loading PUBLIC data for Twin 'prices'
   ✓ Loading data...
   ✓ Loaded: bytes
🌍 Using PUBLIC data from Twin 'prices...'
📢 Published Twin 'prices' (public side available at: /Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/sandbox/kj@kj.dev/datasites/kj@kj.dev/shared/biovault/sessions/49bd18a519de/data/d9eea11b1e5e49b0bea7be1783d7da4d.beaver)


In [16]:
# Verify it's published
session.remote_vars

Name,Type,Status,ID
prices,"Twin[DataFrame] (3, 3)",✓ loaded,f841f7c9ee1b...


## Step 7: Access Peer's Remote Variables

Now let's see how to access data shared by your peer, since we are our own peer, its the same object.

In [17]:
# View what your peer has published
session.peer_remote_vars

Name,Type,ID
prices,"Twin[DataFrame] (3, 3)",f841f7c9ee1b...


In [ ]:
# If peer has published 'prices', load it
# (If you're testing with yourself, you'll see your own data)
# NOTE: Use inject=False to avoid overwriting our local 'prices' Twin
if "prices" in session.peer_remote_vars:
    peer_prices = session.peer_remote_vars["prices"].load(inject=False, trust_loader=True)
    print("Loaded peer's data:")
    print(peer_prices)

## Step 8: Create a Privacy-Aware Function

The `@bv` decorator makes a function **Twin-aware**. When you call it with a Twin, it:
1. First runs on the **public (mock)** data for testing
2. Lets you request execution on the **private (real)** data

In [19]:
peer_prices.public

,item,units,price
0,apple,2,1.40
1,banana,3,0.75
2,orange,1,1.10


In [20]:
@bv
def total_revenue(df: pd.DataFrame) -> float:
    """
    Calculate total revenue from a price DataFrame.
    Revenue = sum(units * price)
    """
    print(f"running on {type(df)}")
    return (df["units"] * df["price"]).sum()

## Step 9: Run on Public Data

First, run the function on the public (mock) data. This is safe - no sensitive data is involved.

In [21]:
peer_prices

🌍 Twin: prices (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public         item  units  price
0   apple      2   1.40
1  banana...    ← .value uses this
  Owner: kj@kj.dev
  Live: ⚫ Disabled
  IDs: twin=2bfc7aef... private=75f5eb48... public=e907e55c...

In [22]:
# Call the function with the Twin
result = total_revenue(peer_prices)

# View the result Twin
result

⚠️  Twin: total_revenue_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━...    ← .value uses this
  🌍 Public    np.float64(6.15)    ✓
  Live: ⚫ Disabled
  📤 Captured stdout: 49 chars
  IDs: twin=53c20f77... private=3d6cf586... public=e63485f8...
  💡 Access: .public_stdout, .public_stderr, .public_figures

In [23]:
result.public

np.float64(6.15)

In [24]:
# Access the public result (mock data)
print(f"Total revenue (mock data): ${result.public:.2f}")

Total revenue (mock data): $6.15


## Step 10: The Private Side Requires Approval

Notice that `.private` doesn't have the answer yet - it shows a computation request instead:

In [25]:
# Private side shows the pending computation
result.private

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: total_revenue_result
   Request from: kj@kj.dev
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: total_revenue

⚠️  WARNING: Function accesses global state!
   These variables are NOT passed as parameters:
   • bv
   • pd
   💡 This may cause different results on remote execution!

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': '2bfc7aef...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=b0e759ebeafa... result=53c20f774b96...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Understanding Twins

A **Twin** is a dual-value container:

```
┌─────────────────────────────────────────┐
│              Twin                       │
├─────────────────┬───────────────────────┤
│  🌍 Public      │  🔒 Private           │
│  (Mock/Safe)    │  (Real/Sensitive)     │
├─────────────────┼───────────────────────┤
│  - Always       │  - Only with          │
│    accessible   │    explicit approval  │
│  - Used for     │  - Contains real      │
│    testing      │    results            │
└─────────────────┴───────────────────────┘
```

When you run a `@bv` decorated function on a Twin:
1. It runs immediately on the **public side**
2. The **private side** becomes a **request** - waiting for approval
3. Once approved, the real result populates `.private`

## Step 11: Request Private Data Execution

To get the real answer, call `.request_private()`. This sends a **computation request** to the data owner.

In [26]:
# Request execution on private data
result.request_private()

📨 Sending computation request to kj@kj.dev
   Function: total_revenue
   Result: result
✓ Sent to /Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/sandbox/kj@kj.dev/datasites/kj@kj.dev/shared/biovault/sessions/49bd18a519de/a7a82e2d46a444f1830e2c9ddb1cf338.beaver
💡 Result will auto-update when kj@kj.dev approves


## Step 12: Check the Session Inbox

The computation request appears in the data owner's inbox. Since we're testing with ourselves, we can see it in our own inbox:

In [27]:
# Check inbox for pending requests
inbox = session.inbox()
inbox

name,id,sender,type,size_bytes,created_at,reply_to
request_total_revenue_for_result,a7a82e2d46a4...,kj@kj.dev,ComputationRequest,793,2025-12-10 16:32:03,


In [28]:
inbox[0]

BeaverEnvelope [CODE]: request_total_revenue_for_result
  From: kj@kj.dev
  Type: ComputationRequest (__main__)
  Size: 793 bytes
  Created: 2025-12-10 16:32:03 UTC
  ID: a7a82e2d...

Arguments (1):
  [0] dict: {'_beaver_twin_ref': True, 'twin_id': '2bfc7aefbba242f294...
      📌 Static value (bound at call time)

Signature: request_total_revenue_for_result(df: pandas.core.frame.DataFrame) -> float

Source:
  def total_revenue(df: pd.DataFrame) -> float:
      """
      Calculate total revenue from a price DataFrame.
      Revenue = sum(units * price)
      """
      print(f"running on {type(df)}")
      return (df["units"] * df["price"]).sum()

Note the code they are asking to run.

In [ ]:
# Load the computation request by name (not index, which varies by platform)
request = inbox["request_total_revenue_for_result"].load(trust_loader=True)
request

When loading variables from the inbox they can define their own symbols in your global space.
This one is called `request_total_revenue_for_result` and so you can immediately use it. If you are already using that variable it will prompt you before overwriting it.

In [29]:
request

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: total_revenue_result
   Request from: client1@sandbox.local
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: total_revenue

⚠️  WARNING: Function accesses global state!
   These variables are NOT passed as parameters:
   • bv
   • pd
   💡 This may cause different results on remote execution!

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': '45c04247...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=cc1797ca4573... result=d91aa0a5ea0d...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 13: Review the Code (As Data Owner)

As the data owner, you can inspect the code before running it on your private data. This is crucial for security - never run code you don't trust!

## Step 14: Test on Mock Data First

Before running on real data, test on mock data to verify the code works correctly:

In [30]:
mock_result = request.run_mock()
print(f"Mock result: {mock_result}")

🧪 Testing on mock/public data...
🔍 [post-execution] Figures to capture: 0
🔍 [post-execution] Already captured IDs: set()
Mock result: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ComputationResult: total_revenue_result
  Request from: kj@kj.dev
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🌍 Twin: total_revenue_result (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public    np.float64(6.15)    ← .value uses this
  Owner: kj@kj.dev
  Live: ⚫ Disabled
  📤 Captured stdout: 49 chars
  IDs: twin=53c20f77... private=877315a5... public=278cc974...
  💡 Access: .public_stdout, .public_stderr, .public_figures

  📤 Output captured during execution:
     running on <class 'pandas.core.frame.DataFrame'>

  💡 Actions:
     .approve_mock()      - Send mock result back
     .reject(message)     - Reject with explanation
     .data                - Access Twin to inspect/modify


## Step 15: Run on Private Data & Approve

If you're satisfied with the code, run it on the real data and approve the result:

In [34]:
private_result = request.run()
print(f"Private result: {private_result}")

# Approve the result - this sends it back to the requester
private_result.approve()

⚙️  Executed: total_revenue_result = total_revenue(...)
   From: kj@kj.dev
   Context: kj@kj.dev
✓ Execution complete
Private result: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ComputationResult: total_revenue_result
  Request from: kj@kj.dev
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔒 Twin: total_revenue_result (REAL DATA - SENSITIVE)
  🔒 Private    np.float64(35.8)    ← .value uses this
  🌍 Public    (not set)
  Live: ⚫ Disabled
  IDs: twin=53c20f77... private=b5012a57... public=0895da2a...

  📤 Output captured during execution:
     running on <class 'pandas.core.frame.DataFrame'>

  💡 Actions:
     .approve()           - Send private/real result back
     .reject(message)     - Reject with explanation
     .data                - Access Twin to inspect/modify
✅ Approving result for: total_revenue_result (twin_id=53c20f774b96..., comp_id=b0e759ebeafa..., result_id=53c20f774b96)
   Sending to: kj@kj.dev
🔒 Using PRIVATE data from Twin 'total_re

PosixPath('/Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/sandbox/kj@kj.dev/datasites/kj@kj.dev/shared/biovault/sessions/49bd18a519de/281b11dd309844c7bdd639402b00f420.beaver')

## Step 16: Receive the Approved Result

The approved result is now available. Load it from inbox and your original `result` variable gets updated!

In [ ]:
# Check inbox for the approved result
inbox = session.inbox()
print(f"Inbox has {len(inbox)} message(s)")

# Load all messages to update our variables
for msg in inbox:
    msg.load(trust_loader=True)

In [36]:
# Now check the private result - it should be updated!
print(f"Total revenue (real data): ${result.private:.2f}")

Total revenue (real data): $35.80


## Summary

Congratulations! You've completed the Beaver Hello World tutorial. Here's what you learned:

1. **Import & Context**: `bv = beaver.ctx()` creates your gateway to Beaver

2. **Sessions**: `session = bv.active_session()` connects you to a peer

3. **Twins**: A `Twin(public=..., private=...)` keeps mock and real data together

4. **Remote Variables**: `session.remote_vars["name"] = twin` shares data (public only!)

5. **@bv Decorator**: Makes functions Twin-aware - runs on public, requests private

6. **Request Flow**:
   - Requester: `result.request_private()` → sends computation request
   - Owner: `request.run()` → `result.approve()` → sends result back
   - Requester: `inbox[...].load()` → updates original variable

### Next Steps

- Try more complex computations
- Work with real datasets (AnnData, etc.)
- Collaborate with actual peers
- Explore the BioVault app for visual dataset management